In [1]:
import cv2
import numpy as np
import _pickle as cPickle
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import os

In [2]:
NUMBER_OF_SIFT_FEATURES = 800
VOCABULARY_SIZE = 128
NUMBER_OF_NEIGHBOURS = 15
DISTANCE_METRIC = 'braycurtis'
DENSE = True
DENSE_STEP_SIZE = 64
DENSE_SIZE = 16

Let us first read the train and test files

In [3]:
train_images_filenames = cPickle.load(open('train_images_filenames.dat','rb'))
test_images_filenames = cPickle.load(open('test_images_filenames.dat','rb'))
train_labels = cPickle.load(open('train_labels.dat','rb'))
test_labels = cPickle.load(open('test_labels.dat','rb'))

In [4]:
train_images_filenames[12]

'../../Databases/MIT_split/train/Opencountry/fie26.jpg'

We create a SIFT object detector and descriptor

In [5]:
if DENSE:
    SIFTdetector = cv2.xfeatures2d.SIFT_create()
else:
    SIFTdetector = cv2.xfeatures2d.SIFT_create(nfeatures=NUMBER_OF_SIFT_FEATURES)

We compute the SIFT descriptors for all the train images and subsequently build a numpy array with all the descriptors stacked together

In [6]:
Train_descriptors = []
Train_label_per_descriptor = []

for filename,labels in zip(train_images_filenames,train_labels):
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    if DENSE:
        kpt = [cv2.KeyPoint(x, y, DENSE_SIZE) for y in range(0, gray.shape[0], DENSE_STEP_SIZE) 
                                              for x in range(0, gray.shape[1], DENSE_STEP_SIZE)]
        kpt,des = SIFTdetector.compute(gray, kpt)
    else:
        kpt,des = SIFTdetector.detectAndCompute(gray,None)
    Train_descriptors.append(des)
    Train_label_per_descriptor.append(labels)

D=np.vstack(Train_descriptors)

We now compute a k-means clustering on the descriptor space

In [7]:
k = VOCABULARY_SIZE
codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
codebook.fit(D)

MiniBatchKMeans(batch_size=2560, compute_labels=False, init='k-means++',
                init_size=None, max_iter=100, max_no_improvement=10,
                n_clusters=128, n_init=3, random_state=42,
                reassignment_ratio=0.0001, tol=0.0, verbose=False)

And, for each train image, we project each keypoint descriptor to its closest visual word. We represent each of the images with the frequency of each visual word.

In [8]:
visual_words=np.zeros((len(Train_descriptors),k),dtype=np.float32)
for i in range(len(Train_descriptors)):
    words=codebook.predict(Train_descriptors[i])
    visual_words[i,:]=np.bincount(words,minlength=k)

We build a k-nn classifier and train it with the train descriptors

In [9]:
knn = KNeighborsClassifier(n_neighbors=NUMBER_OF_NEIGHBOURS,n_jobs=-1,metric=DISTANCE_METRIC)
knn.fit(visual_words, train_labels) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='braycurtis',
                     metric_params=None, n_jobs=-1, n_neighbors=15, p=2,
                     weights='uniform')

We end up computing the test descriptors and compute the accuracy of the model

In [10]:
visual_words_test=np.zeros((len(test_images_filenames),k),dtype=np.float32)
for i in range(len(test_images_filenames)):
    filename=test_images_filenames[i]
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    if DENSE:
        kpt = [cv2.KeyPoint(x, y, DENSE_SIZE) for y in range(0, gray.shape[0], DENSE_STEP_SIZE) 
                                              for x in range(0, gray.shape[1], DENSE_STEP_SIZE)]
        kpt,des = SIFTdetector.compute(gray, kpt)
    else:
        kpt,des=SIFTdetector.detectAndCompute(gray,None)
    words=codebook.predict(des)
    visual_words_test[i,:]=np.bincount(words,minlength=k)

In [11]:
accuracy = 100*knn.score(visual_words_test, test_labels)
print(accuracy)

67.28624535315984


Dimensionality reduction, with PCA and LDA

In [12]:
pca = PCA(n_components=64)
VWpca = pca.fit_transform(visual_words)
knnpca = KNeighborsClassifier(n_neighbors=NUMBER_OF_NEIGHBOURS,n_jobs=-1,metric=DISTANCE_METRIC)
knnpca.fit(VWpca, train_labels) 
vwtestpca = pca.transform(visual_words_test)
accuracy = 100*knnpca.score(vwtestpca, test_labels)
print(accuracy)

68.27757125154895


In [13]:
lda = LinearDiscriminantAnalysis(n_components=64)
VWlda = lda.fit_transform(visual_words,train_labels)
knnlda = KNeighborsClassifier(n_neighbors=NUMBER_OF_NEIGHBOURS,n_jobs=-1,metric=DISTANCE_METRIC)
knnlda.fit(VWlda, train_labels) 
vwtestlda = lda.transform(visual_words_test)
accuracy = 100*knnlda.score(vwtestlda, test_labels)
print(accuracy)

c:\python37\lib\site-packages\sklearn\discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(128, 8 - 1) = 7 components.
  ChangedBehaviorWarning)
c:\python37\lib\site-packages\sklearn\discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


72.36679058240397
